Test and develop units for elasticc, but based on ZTF alerts. 
Content:
    - Grab a small set of alerts (token, through query?).
    - Ingest with the following units scheduled:
    - T2ElasticcRedshiftSampler (print journal, then grab default)
    - T2RunParsnip (based on redshift weigths)
    - T3: ElasticcClassPublisher 

In [ ]:
query = {
  "jd": {
    "$gt": 2459529.5,
    "$lt": 2459530.,
  },
  "candidate": {
    "drb": {
      "$gt": 0.995
    },
    "magpsf": {
      "$gt": 17
    },
    "ndethist": {
      "$gt": 10,
    },
    "isdiffpos": {"$in": ["t", "1"]},
  }
}

In [ ]:
import sys, os, time
import requests
from requests.exceptions import HTTPError
from requests.exceptions import ConnectionError

In [ ]:
from ampel.secret.AmpelVault import AmpelVault
from ampel.secret.DictSecretProvider import DictSecretProvider
from ampel.dev.DevAmpelContext import DevAmpelContext

In [ ]:
# Create a secret vault
secrets = { 'slack/ztf_general/jno': os.environ["SNIaSLACK"],
                              'mongo/logger':{},
                              'mongo/writer':{},
                              'extcats/reader':{},
            'ztf/archive/auth' : {'user' : '', 'password' : ''},
            'datalab/pwd':os.environ["DATALABPWD"],
            "desycloud" : {},
            "ztf/archive/token": os.environ["ARCHIVE_TOKEN"],
           "desc/user": "ampel",
           "desc/password": "manthemachine",
           
          }
vault = AmpelVault([DictSecretProvider(secrets)])

In [ ]:
AMPEL_CONF = '/home/jnordin/github/ampel82/ampel_conf_9587ed.yaml'
channel = 'elasticc'

In [ ]:
ctx = DevAmpelContext.load(
    config = AMPEL_CONF,
    db_prefix = 'dumpme',
    purge_db = False,
    vault = vault
)
ctx.add_channel(
    name=channel,
    access=['ZTF', 'ZTF_PUB', 'ZTF_PRIV']
)

In [ ]:
header = {"Authorization": "bearer "+vault.get_named_secret("ztf/archive/token").value}
endpoint = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3/streams/from_query?programid=1'

In [ ]:
response = requests.post(endpoint, headers=header, json=query )
if not response.ok:
    print( 'Query creation failed' )
    print(response)

In [ ]:
loader_config = {'archive':"https://ampel.zeuthen.desy.de/api/ztf/archive/v3", "stream":response.json()['resume_token']}

In [ ]:
loader_config = {'archive': 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3',
 'stream': 'G708alr46ux7oLutQjTXhj5zaXSyfQmrw0GsVCaHWxY'}

In [ ]:
# These are configuration parameters for the initial alert filter step.
# Note that this filter configuration assumes that a first filter based on nbr of points were made in the alert query
# The T0 stage will not be exposed to the full candidate history. 
filter_config = {
    'min_ndet': 1,
    'min_tspan': 15,
    'max_tspan': 100,
    'min_archive_tspan': 15,
    'max_archive_tspan': 100,    
    'min_rb': 0.3,
    'max_fwhm': 5.5,
    'min_gal_lat': 14,
    'ps1_sgveto_rad': 1,
    'ps1_sgveto_th': 0.8,
    'ps1_confusion_rad': 3,
    'ps1_confusion_sg_tol': 0.1
}

In [ ]:
parsnip_config = {
    'parsnip_model':'/home/jnordin/data/parsnip/parsnip_ps1_ztf.pt', 
    'parsnip_classifier':'/home/jnordin/data/parsnip/ztf_classifier.pkl', 
    "redshift_kind" : "T2ElasticcRedshiftSampler",
    "apply_mwcorrection": True,
    "t2_dependency": [
        {
            "unit": "T2ElasticcRedshiftSampler",
            "link_override": {
                'filter': 'PPSFilter', 'sort': 'jd', "select": "first"
                }
        },     
    ], 
}

In [ ]:
directives = [
        {
            "channel": channel,
            "filter": {
                "unit": "SimpleDecentFilter",
                "config": filter_config,
                "on_stock_match": 'bypass',
            },
            "ingest": {
                "mux": {
                    "unit": "ZiArchiveMuxer",
                    "config": {
                        "history_days": 999,
                        "future_days": 999
                    },
                    "combine": [
                        {
                            "unit": "ZiT1Combiner",
                            "state_t2": [
                                {
                                    "unit": "T2RunParsnip",
                                    "config": parsnip_config,
                                },                            
                            ]
                        }
                    ],
                    "insert": {
                        "point_t2": [
                            {
                                'unit': 'T2ElasticcRedshiftSampler',
                                "ingest": {
                                    'filter': 'PPSFilter', 'sort': 'jd', 'select': 'first'
                                }
                                # For elasticc use a filter to select the DP with diaObject props.
# ingest: =  DPSelection(filter=UnitModel(unit='SimpleTagFilter', config={'require': ['ZTF_DP']}), sort='jd', select='first'}
                            },
                        ],
                    }

                },

            }
            }
        
    ]

In [ ]:
ac = ctx.new_context_unit(
    unit = "AlertConsumer",
    process_name = "AP_test",
    iter_max = 1000,
    log_profile = os.environ.get('log_profile', 'debug'),
    shaper = "ZiDataPointShaper",
    compiler_opts = "ZiCompilerOptions",
    supplier = {
        "unit": "ZiAlertSupplier",
        'config': {
            'deserialize': None,
            'loader': {
                'unit': 'ZTFArchiveAlertLoader',
                'config': loader_config
            }
        }
    },
    directives = directives
)

In [ ]:
n = ac.run()

In [ ]:
n

In [ ]:
t2w = ctx.new_context_unit(
    unit = 'T2Worker',
    process_name = 'T2Processor_6',
    log_profile = os.environ.get('log_profile', 'default')
)

In [ ]:
t2w.run()

In [ ]:
t3config = {
    'desc_user': {'label':'desc/user'},
    'desc_password': {'label':'desc/password'},
    't2classifiers': ['T2RunParsnip'],
}

In [ ]:
t3p = ctx.new_context_unit(
    process_name = "LensPrint",
    unit = "T3Processor",
    execute = [
        {
            "unit": "T3ReviewUnitExecutor",
            "config": {
                "supply": {
                    "unit": "T3DefaultBufferSupplier",
                    "config": {
                        "select": {
                            "unit": "T3StockSelector",
                            "config": {"channel": channel}
                        },
                        "load": {
                            "unit": "T3SimpleDataLoader",
                            "config": {
                                "directives": ["STOCK", "T1", "T2DOC"],
                                "channel": channel
                            }
                        }
                     }
                },
                "stage" : {
                    "unit": "T3SimpleStager",
                    "config": {
                        "execute": [
                            {
                                "unit": "ElasticcClassPublisher",
                                "config": t3config
                            }
                        ]
                    }
                }
            }
        }
    ]
)

In [ ]:
t3p.run()

In [ ]:
ns = vault.get_named_secret('desc/user')

In [ ]:
ns.value